In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import datetime
import math, time
import sklearn
import sklearn.preprocessing
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.layers import Dense, Bidirectional
from keras.layers import LSTM
from keras.preprocessing import sequence
import csv 
import os.path
import gc
import time

C:\Users\Olegok\Anaconda3\envs\py36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:

def create_dataset(min_max_scaler, seq_len = 20, stock='AAPL', margin = 0):
    data = pd.read_csv('stocks/'+stock+'.csv')
    data = data.drop(['direction_up', 'direction_down','Date', 'direction'], axis=1)
    data = data.dropna(axis=0, how='any')
    
    for key in data:
        data[key] = min_max_scaler.fit_transform(data[key].values.reshape(-1,1))
    x_data = data.as_matrix()
    x_data_seq = []
    y_data_seq = []
    for index in range(len(data) - seq_len): 
        x_data_seq.append(x_data[index: index + seq_len])
        y_data_seq.append(x_data[index + seq_len]+margin)
    x_train = np.array(x_data_seq[:math.ceil(len(data)*0.8)][:][:])
    y_train = np.array(y_data_seq[:math.ceil(len(data)*0.8)][:])
    x_test = np.array(x_data_seq[math.ceil(len(data)*0.8):][:][:])
    y_test = np.array(y_data_seq[math.ceil(len(data)*0.8):][:])
    return x_train, y_train, x_test, y_test

In [3]:
def Model(_batch_size, _n_neurons=128 , _dropout=0.5, seq_len=20, _activation='relu'):
    model = Sequential()
    model.add(LSTM(_n_neurons, batch_input_shape=(_batch_size, seq_len, 36)))
    model.add(Dropout(_dropout))
    #model.add(Flatten())
    model.add(Dense(36, activation=_activation)) #relu & signoid
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [4]:
def fit_dataset(batch_size, x_train, y_train, x_test, y_test): 
    x_train = x_train[:(x_train.shape[0]//batch_size)*batch_size]
    y_train = y_train[:(y_train.shape[0]//batch_size)*batch_size]
    x_test = x_test[:(x_test.shape[0]//batch_size)*batch_size]
    y_test = y_test[:(y_test.shape[0]//batch_size)*batch_size]
    return x_train, y_train, x_test, y_test

In [5]:
n_epoch=35
n_neurons = 350
batch = 10
seq = 27
for stock in ['AAPL','MSFT', 'AMZN', 'TSLA', 'GOOGL']:
    for pred_day in [1,3,7]:
        min_max_scaler = sklearn.preprocessing.MinMaxScaler()
        file_name = 'data/'+stock +'-'+ str(pred_day) + '.csv'
        model = Model(batch,n_neurons,0.5, seq)
        x_train, y_train, x_test, y_test = create_dataset(min_max_scaler,seq,stock, pred_day-1 )
        x_train, y_train, x_test, y_test = fit_dataset(batch, x_train, y_train, x_test, y_test)
        model.fit(x_train,
             y_train,
            batch_size=batch,
             epochs=n_epoch,
           validation_data=(x_test, y_test),
            verbose=1)
        score = model.evaluate(x_test, y_test, batch_size=batch)
        test_pred =  model.predict(x_test[x_test.shape[0]-100:], batch_size=batch)
        test_pred = min_max_scaler.inverse_transform(test_pred)
        data = pd.read_csv('stocks/'+stock +'.csv')
        data = data.drop(['direction_up', 'direction_down', 'direction'], axis=1)
        data = data[data.shape[0]-100:].reset_index()
        data["prediction"] = pd.Series(test_pred[:,3])
        data["prediction"] = data["prediction"].shift(pred_day)
        data = data[pred_day:]
        data.to_csv('prediction_try/'+stock +'-'+ str(pred_day) + '.csv', index=False)

Train on 7510 samples, validate on 1860 samples
Epoch 1/1
1860/1860 [==============================] - 2s 871us/step
    index        Date    open    high       low    close      volume  \
90   9379  2018-03-13  182.59  183.50  179.2400  179.970  31168404.0   
91   9380  2018-03-14  180.32  180.52  177.8100  178.440  29075469.0   
92   9381  2018-03-15  178.50  180.24  178.0701  178.650  22584565.0   
93   9382  2018-03-16  178.65  179.12  177.6200  178.020  36836456.0   
94   9383  2018-03-19  177.32  177.47  173.6600  175.300  32804695.0   
95   9384  2018-03-20  175.24  176.80  174.9400  175.240  19314039.0   
96   9385  2018-03-21  175.04  175.09  171.2600  171.270  35247358.0   
97   9386  2018-03-22  170.00  172.68  168.6000  168.845  41051076.0   
98   9387  2018-03-23  168.39  169.92  164.9400  164.940  40248954.0   
99   9388  2018-03-26  168.07  173.10  166.4400  172.770  36272617.0   

        macd     macds     macdh     ...      high_delta  low_delta  \
90  2.749622  2.157

KeyboardInterrupt: 